In [17]:
import findspark
findspark.init()

In [18]:
#from delta import *

In [19]:

# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
import pyspark

#from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

#spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Data
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

# Columns
columns = ["language","users_count"]

# Create DataFrame
df = spark.createDataFrame(data).toDF(*columns)

# Print DataFrame
df.show()


+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+



In [20]:

#from pyspark.sql import SparkSession
#spark=SparkSession.builder.appName("accumulator").getOrCreate()

accum=spark.sparkContext.accumulator(0)
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value) #Accessed by driver


15


In [21]:

# Create spark session with local[5]
rdd = spark.sparkContext.parallelize(range(0,20))
print("From local[5] : "+str(rdd.getNumPartitions()))

# Use parallelize with 6 partitions
rdd1 = spark.sparkContext.parallelize(range(0,25), 6)
print("parallelize : "+str(rdd1.getNumPartitions()))

rddFromFile = spark.sparkContext.textFile(r"C:\sources\azure\pyspark-examples\resources\test.txt",10)
print("TextFile : "+str(rddFromFile.getNumPartitions()))


From local[5] : 8
parallelize : 6
TextFile : 10


In [22]:
from shutil import rmtree
rmtree("c:/data/partition")

In [23]:
rdd1.saveAsTextFile("c:/data/partition")

In [24]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8,9,10])
#rdd = spark.parallelize([1,2,3,4,5,6,7,8,9,10])


In [25]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [26]:
data = ["Project Gutenberg’s",
        "Alice’s Adventures in Wonderland",
        "Project Gutenberg’s",
        "Adventures in Wonderland",
        "Project Gutenberg’s"]
rdd=spark.sparkContext.parallelize(data)
for element in rdd.collect():
    print(element)

Project Gutenberg’s
Alice’s Adventures in Wonderland
Project Gutenberg’s
Adventures in Wonderland
Project Gutenberg’s


In [27]:
for element in rdd.collect():
    print(element)

Project Gutenberg’s
Alice’s Adventures in Wonderland
Project Gutenberg’s
Adventures in Wonderland
Project Gutenberg’s


In [28]:
rdd2=rdd.flatMap(lambda x: x.split(" "))
for element in rdd2.collect():
    print(element)

Project
Gutenberg’s
Alice’s
Adventures
in
Wonderland
Project
Gutenberg’s
Adventures
in
Wonderland
Project
Gutenberg’s


In [29]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])

from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.knownLanguages))
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  null|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [30]:
df.show()

+----------+-------------------+--------------------+
|      name|     knownLanguages|          properties|
+----------+-------------------+--------------------+
|     James|      [Java, Scala]|{eye -> brown, ha...|
|   Michael|[Spark, Java, null]|{eye -> null, hai...|
|    Robert|         [CSharp, ]|{eye -> , hair ->...|
|Washington|               null|                null|
| Jefferson|             [1, 2]|                  {}|
+----------+-------------------+--------------------+



In [31]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("InnerJoinExample").getOrCreate()

# Create DataFrame 1
data1 = [("Alice", 25),
         ("Bob", 30),
         ("Charlie", 28)]
columns1 = ["Name", "Age"]
df1 = spark.createDataFrame(data1, columns1)

# Create DataFrame 2
data2 = [("Alice", "Engineer"),
         ("Charlie", "Designer"),
         ("David", "Analyst")]
columns2 = ["Name", "Occupation"]
df2 = spark.createDataFrame(data2, columns2)

# Perform inner join
result_df = df1.join(df2, on="Name", how="inner")

# Show the result
result_df.show()

# Stop the Spark session
 

+-------+---+----------+
|   Name|Age|Occupation|
+-------+---+----------+
|  Alice| 25|  Engineer|
|Charlie| 28|  Designer|
+-------+---+----------+



In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import col 
data = [("Alice", 25, None),
        ("Bob", None, "Engineer"),
        ("Charlie", 28, "Designer")]
columns = ["Name", "Age", "Occupation"]


df = spark.createDataFrame(data, columns)

# Add a new column using when() function
df2 = df.withColumn("Category", when(df.Age < 30, "Young").otherwise("Adult"))
result_df = df2.withColumn("AgePlusTwo", expr("Age + 2")) \
              .withColumn("AgeSquared", expr("Age * Age")) \
              .withColumn("AgeCategory", expr("CASE WHEN Age < 30 THEN 'Young' ELSE 'Adult' END"))

# Show the result
result_df.show()

+-------+----+----------+--------+----------+----------+-----------+
|   Name| Age|Occupation|Category|AgePlusTwo|AgeSquared|AgeCategory|
+-------+----+----------+--------+----------+----------+-----------+
|  Alice|  25|      null|   Young|        27|       625|      Young|
|    Bob|null|  Engineer|   Adult|      null|      null|      Adult|
|Charlie|  28|  Designer|   Young|        30|       784|      Young|
+-------+----+----------+--------+----------+----------+-----------+



In [39]:
result_df = df.withColumn("NewOccupation", when(col("Occupation").isNull(), "Unknown").otherwise(col("Occupation")))
result_df.show()


+-------+----+----------+-------------+
|   Name| Age|Occupation|NewOccupation|
+-------+----+----------+-------------+
|  Alice|  25|      null|      Unknown|
|    Bob|null|  Engineer|     Engineer|
|Charlie|  28|  Designer|     Designer|
+-------+----+----------+-------------+

